<a href="https://colab.research.google.com/github/kritp144/Sales-Prediction/blob/main/SalesDataPart5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SALES PREDICTION PROJECT
# KRIT PATEL
# 05/18/2022

# THE OBJECTIVE OF THIS PART OF THE PROJECT IS TO CREATE A PRE PROCESSING PIPELINE TO PREPARE THE DATASET FOR MACHINE LEARNING.

IN ORDER TO DO SO, WE MUST FIRST IMPORT ALL RELEVANT LIBRARIES

In [86]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config

# This shows the pipeline diagram
set_config(display= 'diagram')

IMPORTING AND LOADING THE DATA

In [87]:
# Defining the pathway
sales_path = '/content/drive/MyDrive/DATA SCIENCE/CODES/project 1/sales_predictions.csv'

# Defining the DatatFlow and Reading the Data
sales_df = pd.read_csv(sales_path)
sales_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [88]:
# Checkin the info 
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [89]:
# Standardize the Item_Fat_Content Data

# Replacing LF with Low Fat
sales_df['Item_Fat_Content'] = sales_df['Item_Fat_Content'].replace('LF', 'Low Fat')

# Replacing low fat to Low Fat
sales_df['Item_Fat_Content'] = sales_df['Item_Fat_Content'].replace('low fat', 'Low Fat')

# Replacing reg to Regular

sales_df['Item_Fat_Content'] = sales_df['Item_Fat_Content'].replace('reg', 'Regular')



IDENTIFING THE TARGET (Y) AND THE FEATURES (X)

In [90]:
X = sales_df.drop(columns= ['Item_Outlet_Sales'])
y = sales_df['Item_Outlet_Sales']  

PERFORMING A TRAIN TEST SPLIT

In [91]:
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=42) 


CREATING A PRE PROCESSING PIPELINE

1) IDENTIFYING EACH FEATURE:


NUMERICAL FEATURES: Item_Weight, Item_Visibility, Item_MRP, Outlet_Establishment_Year


ORDINAL FEATURES: Outlet_Size, Outlet_Tier, Item_Fat_Content 


NORMINAL FEATURES: Item_Type, Outlet_Type

In [92]:
# Instantiating Column Selectors

#Categorical Column Selector
cat_selector = make_column_selector(dtype_include= 'object')

# Numerical Column Selector
num_selector = make_column_selector(dtype_include= 'number')

In [93]:
# Instantiating Transformers

# Frequency Imputers
freq_imputer = SimpleImputer(strategy= 'most_frequent')

# Mean Imputer
mean_imputer = SimpleImputer(strategy= 'mean')

# Scaler
scaler= StandardScaler()

# One Hot Encoder
ohe = OneHotEncoder(sparse= False, handle_unknown= 'ignore')

In [94]:
#Instantiating Pipelines

# Numerical Pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe 

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [95]:
from pandas.core.arrays import categorical
# Categorical Pipwline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [96]:
# Instantiating Column Transformer

#Tuples for Coulumn Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)

# Column Transformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f80939ca950>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f80939ca550>)])

In [97]:
# Fit the training data
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f80939ca950>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f80939ca550>)])

In [98]:
# Transforning Training and Testing Data
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)